<a href="https://colab.research.google.com/github/PolinaLinaNechaeva/lfs-crm-analytics/blob/main/master_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. Read spend file
data_spend = pd.read_excel("Spend (Done).xlsx")
print(data_spend.head(5))
print(data_spend.tail(5))

data_spend.info()

# bring Date to unified format (dd.mm.yyyy), NaT → 31.12.2099
if "Date" in data_spend.columns:
    parsed = pd.to_datetime(data_spend["Date"], errors="coerce", dayfirst=True)

    if parsed.isna().all():
        # try another format if everything failed
        parsed = pd.to_datetime(
            data_spend["Date"],
            errors="coerce",
            format="%Y-%m-%d %H:%M:%S",
        )

    parsed = parsed.fillna(pd.to_datetime("2099-12-31"))
    data_spend["Date"] = parsed.dt.strftime("%d.%m.%Y")

# check
print(data_spend["Date"].head(10))
print("NaN remaining:", data_spend["Date"].isna().sum())

# percentage of missing values in each column
nan_percent = (data_spend.isna().sum() / len(data_spend) * 100).round(2)
print(nan_percent)

data_spend["Campaign"] = data_spend["Campaign"].fillna("")
data_spend["AdGroup"] = data_spend["AdGroup"].fillna("")
data_spend["Ad"] = data_spend["Ad"].fillna("")
nan_percent = (data_spend.isna().sum() / len(data_spend) * 100).round(2)
print(nan_percent)

# Find and remove full duplicates
dups_full = data_spend[data_spend.duplicated(keep=False)]

# all full duplicates
print(dups_full.head(10))
print(dups_full.tail(10))

print(f"Full duplicates found: {dups_full.shape[0]}")

before = len(data_spend)
# remove full duplicate rows
data_spend = data_spend.drop_duplicates()
after = len(data_spend)

print(f"Removed {before - after} full duplicates. Rows left: {after}")

# save cleaned dataframe
data_spend.to_excel("data_spend_clean.xlsx", index=False)

print("File saved successfully: data_spend_clean.xlsx")

"""## Deals Cleaning"""

# 1. Read deals
data_deals = pd.read_excel("DWW Deals Done.xlsx")
print(data_deals.head(5))
print(data_deals.tail(5))

data_deals.info()

# Ranges for numerical fields
numeric_cols = data_deals.select_dtypes(include=[np.number]).columns
# ranges for all numeric columns
for col in numeric_cols:
    col_min = data_deals[col].min()
    col_max = data_deals[col].max()
    print(f"{col}: min = {col_min}, max = {col_max}")

print("Ranges for numeric fields:")

# Check free columns
nan_percent_deals = (data_deals.isna().sum() / len(data_deals) * 100).round(2)
print(nan_percent_deals)

# Work with ID
data_deals["DEALID"] = data_deals["DEALID"].astype(str)

# Work with Amount
data_deals["Amount"] = pd.to_numeric(data_deals["Amount"], errors="coerce")
data_deals["Amount"].describe()

# Work with pipeline
data_deals["PIPELINE"] = data_deals["PIPELINE"].astype(str)

# Work with Stage
data_deals["STAGE"] = data_deals["STAGE"].astype(str)

# Work with Lost Reason
data_deals["Lost Reason"] = data_deals["Lost Reason"].astype(str)

# Work with Created Date, Closing Date
data_deals["Created Date"] = pd.to_datetime(
    data_deals["Created Date"], errors="coerce", dayfirst=True
)
data_deals["Closing Date"] = pd.to_datetime(
    data_deals["Closing Date"], errors="coerce", dayfirst=True
)

nan_percent_deals = (data_deals.isna().sum() / len(data_deals) * 100).round(2)
print(nan_percent_deals)

# Delete rows with Lost Reason = 'Duplicate'
data_deals = data_deals[data_deals["Lost Reason"] != "Duplicate"]

# check result
print(data_deals["Lost Reason"].value_counts(dropna=False))

print(f"Rows left: {len(data_deals)}")

# Replace NaN in Closing Date with 1900-01-01
data_deals["Closing Date"] = data_deals["Closing Date"].fillna(
    pd.to_datetime("1900-01-01")
)

# Replace NaN in SLA with mode
if "SLA" in data_deals.columns:
    sla_mode = data_deals["SLA"].mode(dropna=True)
    if not sla_mode.empty:
        data_deals["SLA"] = data_deals["SLA"].fillna(sla_mode.iloc[0])

# For numeric columns: NaN → 0
num_cols = data_deals.select_dtypes(include=[np.number]).columns
data_deals[num_cols] = data_deals[num_cols].fillna(0)

# For string columns: NaN → 'Unknown'
str_cols = data_deals.select_dtypes(include=["object"]).columns
data_deals[str_cols] = data_deals[str_cols].fillna("Unknown")

print("Missing values handled:")
print(data_deals.isna().sum().sum(), "NaN remaining")

unique_reasons = data_deals["Lost Reason"].unique()
print("Unique reasons:", unique_reasons)

# create new column with grouped reasons
mapping = {
    "price": [
        "Cheaper provider",
        "Price",
        "Price/Costs",
        "budget",
    ],
    "internal": [
        "Internal HR",
        "Internal Recruiter",
        "Inhouse HR",
        "Inhouse recruiting",
        "Internal restructuring",
    ],
    "no decision": [
        "No decision",
        "Not now",
        "Not the right time",
        "Not ready",
    ],
    "other": [
        "Other reasons",
        "Unknown",
        "Refer to competitor",
    ],
}

def categorize_reason(reason: str) -> str:
    reason_lower = str(reason).lower()
    for category, keywords in mapping.items():
        for kw in keywords:
            if kw.lower() in reason_lower:
                return category
    return "other"


data_deals["Lost Reason Grouped"] = data_deals["Lost Reason"].apply(
    categorize_reason
)

# check distribution
print(data_deals["Lost Reason Grouped"].value_counts(dropna=False))

# search for full duplicates
dups_deals = data_deals[data_deals.duplicated(keep=False)]
print(dups_deals.head(10))

print(f"Total rows in table: {len(data_deals)}")
print(
    f"Number of rows that are part of full duplicates: {len(dups_deals)}"
)
print(
    f"Number of unique full duplicates: {dups_deals.duplicated().sum()}"
)

# remove duplicates and keep dataframe in variable
data_deals_dedup = data_deals.drop_duplicates()

# save separately
data_deals_dedup.to_excel("DWW Deals clean.xlsx", index=False)

# save cleaned dataframe
data_deals.to_excel("data_deals_clean.xlsx", index=False)

print("File saved successfully: data_deals_clean.xlsx")

"""## Calls Cleaning"""

data_calls = pd.read_excel("DWW Calls Done.xlsx")
print(data_calls.head())
print(data_calls.info())

# Fix scientific notation in Duration if needed
def normalize_duration(x):
    """
    Normalize call duration values that might be in scientific notation or as strings.
    """
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float)):
        return x
    if isinstance(x, str):
        # expand exponent form
        if "e" in x.lower():
            try:
                return float(x)
            except ValueError:
                pass
        x = x.replace("sec", "").replace("s", "").strip()
        x = x.replace(",", ".")
        x = x.replace(" ", "")
        x = x.replace(".", "")  # remove dots
        # keep only one prefix
        if x.startswith("00:"):
            x = x[3:]
        try:
            return float(x)
        except ValueError:
            return np.nan
    return np.nan


if "Duration" in data_calls.columns:
    data_calls["Duration_norm"] = data_calls["Duration"].apply(normalize_duration)

    print(data_calls[["Duration", "Duration_norm"]].head(20))

    # drop original column
    data_calls = data_calls.drop(columns=["Duration"])
    data_calls = data_calls.rename(columns={"Duration_norm": "Duration"})

print(data_calls.info())

# Convert Call Start Time to datetime
data_calls["Call Start Time"] = pd.to_datetime(
    data_calls["Call Start Time"], errors="coerce"
)

# create date-only column
data_calls["Call Date"] = data_calls["Call Start Time"].dt.date

# replace Call Start Time with time only
data_calls["Call Time"] = data_calls["Call Start Time"].dt.time

# check
print(data_calls[["Call Start Time", "Call Date", "Call Time"]].head(10))

# Fill missing CONTACTID by mode per owner
if "CONTACTID" in data_calls.columns and "OWNER" in data_calls.columns:
    # 2) fill CONTACTID with mode per call owner
    mode_per_owner = (
        data_calls.groupby("OWNER")["CONTACTID"]
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    )
    data_calls["CONTACTID"] = data_calls.apply(
        lambda row: mode_per_owner[row["OWNER"]]
        if pd.isna(row["CONTACTID"]) and row["OWNER"] in mode_per_owner
        else row["CONTACTID"],
        axis=1,
    )

# 3) fill duration with mode per call owner
if "Duration" in data_calls.columns and "OWNER" in data_calls.columns:
    duration_mode_per_owner = (
        data_calls.groupby("OWNER")["Duration"]
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    )
    data_calls["Duration"] = data_calls.apply(
        lambda row: duration_mode_per_owner[row["OWNER"]]
        if pd.isna(row["Duration"]) and row["OWNER"] in duration_mode_per_owner
        else row["Duration"],
        axis=1,
    )

# Unique statuses
unique_statuses = data_calls["Call Status"].unique()
print("Unique statuses:", unique_statuses)

# Keep only Completed calls
data_calls = data_calls[data_calls["Call Status"] == "Completed"]

print(f"Rows left: {len(data_calls)}")

# analyze call duration
sns.boxplot(x=data_calls["Duration"])
plt.title("Call duration boxplot (seconds)")
plt.xlabel("Duration (sec)")
plt.show()

# number of full duplicates (all columns)
dups_calls_full = data_calls[data_calls.duplicated(keep=False)]
print(dups_calls_full.head(10))

# what if calls happened simultaneously from two devices?
# check overlapping calls per manager
group_cols = ["OWNER", "Call Date", "Call Time"]
data_calls["calls_at_same_time"] = data_calls.groupby(group_cols)["Call Time"].transform(
    "count"
)

parallel_calls = (
    data_calls[data_calls["calls_at_same_time"] > 1]
    .sort_values(group_cols + ["Duration"])
    .query("calls_at_same_time > 1")   # keep only simultaneous
)

print("Cases where a manager had several calls at the same time:")
print(parallel_calls.head(20))

# if a call has an exact duplicate (everything same except Id and duration) —
# keep only one, with duration closest to that manager's median

# 0) make sure duration is numeric
data_calls["Duration"] = pd.to_numeric(data_calls["Duration"], errors="coerce")

# 1) duration medians per manager
medians_by_owner = data_calls.groupby("OWNER")["Duration"].median()

# 2) grouping columns: everything except Id and duration
group_cols = [c for c in data_calls.columns if c not in ["Id", "Duration"]]

# 3) function to choose row index whose duration is closest to manager median
def pick_best_index(group: pd.DataFrame) -> int:
    owner = group["OWNER"].iloc[0]
    median_owner = medians_by_owner.get(owner, np.nan)

    # if manager median is undefined (all NaN), use group median
    if pd.isna(median_owner):
        median_group = group["Duration"].median()
    else:
        median_group = median_owner

    # if that also fails (all NaN), return first row in group
    if pd.isna(median_group):
        return group.index[0]

    # distance to median (replace NaN with large number to avoid choosing them)
    distances = (group["Duration"] - median_group).abs().fillna(1e9)

    # index of row with minimal deviation
    best_idx = distances.idxmin()
    return best_idx


# 4) pick best index from each group
best_indices = []
for _, g in data_calls.groupby(group_cols):
    if len(g) == 1:
        best_indices.append(g.index[0])
    else:
        best_indices.append(pick_best_index(g))

best_indices = pd.Index(best_indices)

# if Series with MultiIndex is returned — take index values
best_indices = best_indices.unique()

# 5) build deduplicated dataframe
before = len(data_calls)
data_calls_dedup = data_calls.loc[best_indices].copy()
after = len(data_calls_dedup)

print(f"Before: {before} rows")
print(f"After deduplication: {after} rows")
print(f"Removed overlapping calls: {before - after}")

sns.boxplot(x=data_calls_dedup["Duration"])
plt.title("Call duration boxplot (seconds)")
plt.xlabel("Duration (sec)")
plt.show()

# see how to bin calls by duration
duration_sec = pd.to_numeric(data_calls_dedup["Duration"], errors="coerce")

# 2) create duration-in-minutes column
duration_min = duration_sec / 60.0

# 3) histogram + category boundaries
plt.hist(duration_min, bins=50)
for x in [2, 5, 15]:
    plt.axvline(x, color="red", linestyle="--")

plt.xlim(0, 60)  # to see the main mass
plt.title("Distribution of call duration (minutes)")
plt.xlabel("Call duration, min")
plt.ylabel("Number of calls")
plt.show()

# our boundaries
labels = ["short <2", "medium 2–5", "long 5–15", "super long >15"]


def categorize_duration(x: float) -> str:
    if x < 2:
        return "short (<2)"
    if x < 5:
        return "medium (2–5)"
    if x < 15:
        return "long (5–15)"
    return "super long (>15)"


order = ["short (<2)", "medium (2–5)", "long (5–15)", "super long (>15)"]

data_calls_dedup["Duration_min"] = duration_min
data_calls_dedup["Duration_cat"] = data_calls_dedup["Duration_min"].apply(
    categorize_duration
)

sns.boxplot(
    x="Duration_min",
    y="Duration_cat",
    data=data_calls_dedup,
    order=order,
    showfliers=True   # show outliers as dots
)
plt.title("Call duration boxplot by category")
plt.xlabel("Call duration, min")
plt.ylabel("Category")
plt.show()

# compute outlier bounds for each category
bounds = {}
for cat in order:
    vals = data_calls_dedup.loc[
        data_calls_dedup["Duration_cat"] == cat, "Duration_min"
    ].dropna()
    if len(vals) == 0:
        bounds[cat] = (np.nan, np.nan)
    else:
        q1 = vals.quantile(0.25)
        q3 = vals.quantile(0.75)
        iqr = q3 - q1
        low = q1 - 1.5 * iqr
        high = q3 + 1.5 * iqr
        bounds[cat] = (low, high)

print("Statistics and outlier bounds:")
for cat, (low, high) in bounds.items():
    print(f"{cat}: low={low}, high={high}")

# convert to numbers and minutes
bounds_min = {
    cat: (float(low), float(high))
    for cat, (low, high) in bounds.items()
    if not (np.isnan(low) or np.isnan(high))
}

# categories (in minutes)
def is_valid_duration(row):
    cat = row["Duration_cat"]
    val = row["Duration_min"]
    if cat not in bounds_min:
        return False
    low, high = bounds_min[cat]
    return low <= val <= high


# propagate bounds to rows
data_calls_dedup["is_valid"] = data_calls_dedup.apply(is_valid_duration, axis=1)

# validity: within bounds and not NaN
before = len(data_calls_dedup)
data_calls_clean = data_calls_dedup[
    data_calls_dedup["is_valid"] & data_calls_dedup["Duration_min"].notna()
]
after = len(data_calls_clean)

print(f"Before cleaning: {before}")
print(f"After removing outliers: {after}")
print(f"Removed: {before - after}")

fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(
    x="Duration_min",
    y="Duration_cat",
    data=data_calls_clean,
    order=order,
    ax=ax,
)
ax.set_title("Call duration boxplot by category")
ax.set_xlabel("Call duration, min")
ax.set_ylabel("Category")

plt.show()

# save cleaned dataframe
data_calls_clean.to_excel("data_calls_clean.xlsx", index=False)

print("File saved successfully: data_calls_clean.xlsx")

"""## Contacts Cleaning"""

data_contacts = pd.read_excel("DWW Contacts Done.xlsx")
print(data_contacts.head())
print(data_contacts.info())

# convert Created Time and Modified Time to datetime
datetime_cols = ["Created Time", "Modified Time"]
for col in datetime_cols:
    if col in data_contacts.columns:
        data_contacts[col] = pd.to_datetime(
            data_contacts[col], errors="coerce"
        )

# create new columns
for base_col in datetime_cols:
    if base_col in data_contacts.columns:
        data_contacts[f"{base_col} Date"] = data_contacts[base_col].dt.date
        data_contacts[f"{base_col} Time"] = data_contacts[base_col].dt.time

# check
print(
    data_contacts[
        ["Created Time", "Created Time Date", "Created Time Time"]
    ].head(10)
)

# transform Id: cast to string and prefix with "A"
if "CONTACTID" in data_contacts.columns:
    data_contacts["CONTACTID"] = data_contacts["CONTACTID"].astype(str)
    data_contacts["CONTACTID"] = data_contacts["CONTACTID"].apply(
        lambda x: "A" + x if x.strip() != "" else x
    )

# find all full duplicates (all columns equal)
dups_contacts = data_contacts[data_contacts.duplicated(keep=False)]

print(f"Total rows in table: {len(data_contacts)}")
print(
    f"Number of rows that are part of full duplicates: {len(dups_contacts)}"
)
print(
    f"Number of unique full duplicates: {dups_contacts.duplicated().sum()}"
)

# show several examples of duplicates
print(dups_contacts.head(10))

# number of NaN per column
nan_percent_contacts = (
    data_contacts.isna().sum() / len(data_contacts) * 100
).round(2)
print("Missing values in data_contacts:")
print(nan_percent_contacts)

# save cleaned dataframe
data_contacts.to_excel("data_contacts_clean.xlsx", index=False)

print("File saved successfully: data_contacts_clean.xlsx")

# since Google Sheets refuses to read columns in correct type, use a workaround
def add_prefix_A(col: pd.Series) -> pd.Series:
    col = col.astype(str).fillna("")                # replace NaN with empty string
    col = col.astype(str)               # to string
    col = col.str.replace(".", "", regex=False)  # remove dots
    return col.where(col.eq(""), "A" + col)   # keep empty as "", prepend "A" to others


"""## Final assembly into a single Excel file"""

def safe_read_excel(path: str, var_name: str, fallback_file: str):
    """
    Try to read from existing dataframe (if defined), otherwise from file.
    """
    if var_name in globals():
        return globals()[var_name]
    if os.path.exists(path):
        try:
            return pd.read_excel(path)
        except Exception as e:
            raise RuntimeError(f"Failed to read {p}: {e}")
    raise RuntimeError(f"Neither dataframe '{var_name}' nor file '{fallback_file}' found.")


data_spend_final = safe_read_excel("data_spend_clean.xlsx", "data_spend", "Spend (Done).xlsx")
data_deals_final = safe_read_excel("data_deals_clean.xlsx", "data_deals", "DWW Deals Done.xlsx")
data_calls_final = safe_read_excel("data_calls_clean.xlsx", "data_calls_clean", "DWW Calls Done.xlsx")
data_contacts_final = safe_read_excel("data_contacts_clean.xlsx", "data_contacts", "DWW Contacts Done.xlsx")

with pd.ExcelWriter("projekt.xlsx") as writer:
    data_spend_final.to_excel(writer, sheet_name="spend", index=False)
    data_deals_final.to_excel(writer, sheet_name="deals", index=False)
    data_calls_final.to_excel(writer, sheet_name="calls", index=False)
    data_contacts_final.to_excel(writer, sheet_name="contacts", index=False)

print("File saved successfully: projekt.xlsx (sheets: spend, deals, calls, contacts)")